In [6]:
import pandas as pd
import numpy as np
from tqdm import tqdm
!pip install openrouteservice
import openrouteservice

In [255]:
files_dire = '../data/curated/'
df = pd.read_csv(f'{files_dire}/processed_listing.csv').iloc[: , 1:]
df.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01
1,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2016-12-01
2,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2016-01-01
3,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0,2015-11-01
4,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01


In [256]:
df['list_date'] = df['list_date'].apply(pd.to_datetime)

In [257]:
n_rows = len(df)

In [258]:
len(list(set(df['address'].tolist())))

82556

In [259]:
df2 = df.iloc[0:1,:]
df2 = df2.assign(list_history = '')
df2.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date,list_history
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01,


In [260]:
n_property = 0

In [261]:
past_listing = []
list_date = df.iloc[0]['list_date']

In [262]:
for i in tqdm(range(1,n_rows)):
    # pprint(i)
    if df.iloc[i]['address'] == df2.iloc[n_property]['address']:
        days = (df2.iloc[n_property]['list_date'] - list_date)/ np.timedelta64(1, 'Y')
        list_date = df2.iloc[n_property]['list_date']
        price = df2.iloc[n_property]['weekly_rent'] - df.iloc[i]['weekly_rent']
        if days != 0:
            temp = [days, price]
            past_listing.append(temp)
    else:
        df2.at[n_property, 'list_history'] = past_listing
        n_property += 1
        df2.loc[n_property] = df.iloc[i]
        # df2 = df2.append(df.iloc[i])
        # print(past_listing)
        # print(past_listing)
        past_listing = []
df2.at[n_property, 'list_history'] = past_listing

100%|██████████| 303639/303639 [07:25<00:00, 682.13it/s]


In [263]:
df2.head()

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date,list_history
0,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0,2016-12-01,[]
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01,"[[4.084957254426853, -10.0]]"
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,220.0,2008-10-01,"[[-12.252133856273572, 40.0]]"
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,320.0,2021-01-01,"[[12.252133856273572, 50.0]]"
4,3,2,2,House,"1 YORK CT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"York Court, Karingal, Frankston, Melbourne, Ci...",-38.139321,145.157489,400.0,2021-01-01,[]


In [265]:
len(df)

303640

In [264]:
df2[df2['list_history'].map(lambda d: len(d)) > 0]

,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent,list_date,list_history
1,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0,2021-01-01,"[[4.084957254426853, -10.0]]"
2,3,1,2,AUF,"1/14 PARER STREET, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,220.0,2008-10-01,"[[-12.252133856273572, 40.0]]"
3,3,1,2,AUF,"1/14 PARER ST, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Parer Street, Frankston, Melbourne, City of Fr...",-38.143728,145.140240,320.0,2021-01-01,"[[12.252133856273572, 50.0]]"
5,4,1,2,House,"8 MERLIN COURT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Merlin Court, Karingal, Frankston, Melbourne, ...",-38.141709,145.153106,400.0,2017-11-01,"[[-3.1677584070857034, 0.0]]"
6,3,2,1,Townhouse,"1/2 GORDON AVENUE, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Gordon Avenue, Frankston, Melbourne, City of F...",-38.143041,145.141483,450.0,2020-02-01,"[[2.2505595597445534, -19.999999999999943]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82548,2,1,1,AUF,"5/198 GRANGE RD, CARNEGIE",Carnegie,3163,https://www.oldlistings.com.au/real-estate/VIC...,"Grange Road, Carnegie, Melbourne, City of Glen...",-37.881444,145.049445,310.0,2021-11-01,"[[1.251223502193748, 0.0]]"
82550,3,2,1,NaN,"7/7 MAROONA ROAD, CARNEGIE",Carnegie,3163,https://www.oldlistings.com.au/real-estate/VIC...,"Maroona Road, Carnegie, Melbourne, City of Gle...",-37.890048,145.047566,550.0,2019-08-01,"[[-2.2532974667515417, 30.0]]"
82551,1,1,1,AUF,"116/9 MORTON AVENUE, CARNEGIE",Carnegie,3163,https://www.oldlistings.com.au/real-estate/VIC...,"Morton Avenue, Carnegie, Melbourne, City of Gl...",-37.886487,145.058239,350.0,2021-11-01,"[[2.2532974667515417, -40.0]]"
82554,2,1,1,NaN,"7/4 KOKARIBB ROAD, CARNEGIE",Carnegie,3163,https://www.oldlistings.com.au/real-estate/VIC...,"Kokaribb Road, Carnegie, Melbourne, City of Gl...",-37.887325,145.056148,395.0,2015-01-01,"[[-6.833815889443315, 60.0]]"


In [267]:
df2.groupby(len('list_history')).address.transform('count')

KeyError: 12